*This analysis goes over two very helpful techniques:*

1. The use of commands such as re.search as part of the RegEx expressions to extract values from strings.
2. Fuzzy String Matching.

Data Source: Extracted from Google APIs, Big_query 
Dataset name: San_francisco_bikeshare.

This data provides information about Bike Share Trips in San Francisco.

First: importing required environments.

In [ ]:
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import fuzz,process

Second: reading cvs dataset.

In [2]:
df = pd.read_csv('SFbikesharetrips.csv', encoding='latin1')

Third: examining data and variables.

In [3]:
df.head(2)

,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,start_station_latitude,start_station_longitude,end_station_latitude,end_station_longitude,member_birth_year,member_gender
0,2018-03-31 22:13:30 UTC,Post St at Kearny St,19,2018-03-31 22:34:35 UTC,Broderick St at Oak St,71,1021,37.788975,-122.403452,37.773063,-122.439078,1993.0,Female
1,2018-03-31 21:18:59 UTC,Valencia St at Cesar Chavez St,141,2018-03-31 21:24:24 UTC,Valencia St at 22nd St,133,1082,37.747998,-122.420219,37.755213,-122.420975,1992.0,Male


In [4]:
df.shape

(1000, 13)

In [5]:
df.columns

Index([u'start_date', u'start_station_name', u'start_station_id', u'end_date',
       u'end_station_name', u'end_station_id', u'bike_number',
       u'start_station_latitude', u'start_station_longitude',
       u'end_station_latitude', u'end_station_longitude', u'member_birth_year',
       u'member_gender'],
      dtype='object')




Fourth: examining pick-up and drop-off addresses.

In [6]:
dfstartaddress = df['start_station_name']

In [7]:
dfstartaddress.head(5)

0              Post St at Kearny St
1    Valencia St at Cesar Chavez St
2         Fulton St at Bancroft Way
3        S. 4th St at San Carlos St
4          Telegraph Ave at 27th St
Name: start_station_name, dtype: object

Fifth: identifying main address types.

In [8]:
pick_up_address = []

for i in dfstartaddress:
    if re.search('St|Way|Ave|5th at Folsom|Broadway at Kearny', i):
        pick_up_address.append('Street Address')
    elif re.search('BART', i):
        pick_up_address.append('Bart Station Address')
    elif re.search('Park|Hall|Center|Square|Playground|Plaza|Public Market', i):
        pick_up_address.append('Public Place Address')
    else:
        pick_up_address.append('')

In [9]:
df['pick_up_address'] = pick_up_address

Sixth: making sure all address types have been identified.

In [10]:
df.isnull().sum()

start_date                  0
start_station_name          0
start_station_id            0
end_date                    0
end_station_name            0
end_station_id              0
bike_number                 0
start_station_latitude      0
start_station_longitude     0
end_station_latitude        0
end_station_longitude       0
member_birth_year          63
member_gender              63
pick_up_address             0
dtype: int64

Seventh: examining drop-off address.

In [11]:
dfendaddress = df['end_station_name']

In [12]:
dfendaddress.head(5)

0           Broderick St at Oak St
1           Valencia St at 22nd St
2                45th St at Manila
3        San Salvador St at 9th St
4    Shattuck Ave at Telegraph Ave
Name: end_station_name, dtype: object

Eighth: identifyng main address types.

In [13]:
drop_off_address = []

for a in dfendaddress:
    if re.search('St|Way|Ave|5th at Folsom|Broadway at Kearny', a):
        drop_off_address.append('Street Address')
    elif re.search('BART', a):
        drop_off_address.append('Bart Station Address')
    elif re.search('Park|Hall|Center|Square|Playground|Plaza|Public Market', a):
        drop_off_address.append('Public Place Address')
    else:
        drop_off_address.append('')

In [14]:
df['drop_off_address'] = drop_off_address

Nineth: making sure all addresses types have been identified.

In [15]:
df.isnull().sum()

start_date                  0
start_station_name          0
start_station_id            0
end_date                    0
end_station_name            0
end_station_id              0
bike_number                 0
start_station_latitude      0
start_station_longitude     0
end_station_latitude        0
end_station_longitude       0
member_birth_year          63
member_gender              63
pick_up_address             0
drop_off_address            0
dtype: int64

Tenth: FuzzyWuzzy,
    setting ratios with pick-up and drop-off addresses, through the 
    application of token set ratios    

In [16]:
def get_ratio(row):
    name = row['pick_up_address']
    namel = row['drop_off_address']
    return fuzz.token_set_ratio(name, namel)

Restricting the data to the pairs that exceeded a match score of 65.

In [17]:
df[df.apply(get_ratio, axis=1) > 65].head(5)

,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,start_station_latitude,start_station_longitude,end_station_latitude,end_station_longitude,member_birth_year,member_gender,pick_up_address,drop_off_address
0,2018-03-31 22:13:30 UTC,Post St at Kearny St,19,2018-03-31 22:34:35 UTC,Broderick St at Oak St,71,1021,37.788975,-122.403452,37.773063,-122.439078,1993.0,Female,Street Address,Street Address
1,2018-03-31 21:18:59 UTC,Valencia St at Cesar Chavez St,141,2018-03-31 21:24:24 UTC,Valencia St at 22nd St,133,1082,37.747998,-122.420219,37.755213,-122.420975,1992.0,Male,Street Address,Street Address
2,2018-03-31 20:24:12 UTC,Fulton St at Bancroft Way,247,2018-03-31 21:21:09 UTC,45th St at Manila,210,1038,37.867789,-122.265896,37.833294,-122.256224,1964.0,Male,Street Address,Street Address
3,2018-03-31 19:46:11 UTC,S. 4th St at San Carlos St,208,2018-03-31 19:51:35 UTC,San Salvador St at 9th St,317,2731,37.332836,-121.883912,37.333955,-121.877349,1992.0,Female,Street Address,Street Address
4,2018-03-31 17:18:07 UTC,Telegraph Ave at 27th St,179,2018-03-31 19:03:30 UTC,Shattuck Ave at Telegraph Ave,158,3208,37.816073,-122.267886,37.833279,-122.263490,NaN,NaN,Street Address,Street Address


In [18]:
len(df[df.apply(get_ratio, axis=1) > 65])/len(df)

1

Over 50% of the pairs exceeded a match score of 65.

Eleventh: exporting dataset to csv.

In [19]:
df.to_csv(r'SFBiketrips.cvs', index = False)